<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/3_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split


for dirname, _, filenames in os.walk('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Mounted at /content/drive
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/train_transaction.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/train_identity.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/original_data.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/num_sign_col.txt
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/num_sign_col_mice.txt
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/cat_sign_col.txt
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/corr_data.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted_old/fitted.csv
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted_old/cat_sign_col.txt
/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted_old/num_sign_col_mice.txt
/content/drive/MyDrive/Tesi_magistrale/Dataset/I

In [2]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

Riferimento: https://stackoverflow.com/questions/30483246/how-to-check-if-a-python-module-has-been-imported

# Preprocessing

## Dataset merging

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # serve per ignorare i FutureWarning

Rimpiazzo i missing values in categorical data con unknown. 

Unisco i dataset numerici e categorici nel dataframe data. 

Riferimento fillna: https://jamesrledoux.com/code/imputation

In [4]:
if not os.path.isfile('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv'):
  col_to_drop = col_not_sign(categorical_data, cat_sign_col)
  categorical_data = categorical_data.drop(col_to_drop, axis=1)
  categorical_data = categorical_data.fillna('unknown')
  data = pd.merge(correlation_data, categorical_data, left_on='TransactionID', right_on='TransactionID', how='left')
  del correlation_data, categorical_data, numerical_data
  data.to_csv(r'./data.csv')

data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [5]:
fraud = (data['isFraud'] == 1).sum()
safe = (data['isFraud'] == 0).sum()
ratio = np.ceil(safe/fraud)

## One hot encoding

One hot encoding delle features categoriche. Serve per lo smote e easy ensemble. 

Bisogna farlo prima dello split del dataset poichè rischio di perdere delle colonne. 

In [6]:
data = pd.get_dummies(data)

In [7]:
cols = mf.get_col(data)

Seleziono i dati appartenenti ai primi 60 giorni. 

Se ne seleziono di più, l'easy ensemble non va. 

In [8]:
data = mf.select_days(data, 60)

Splitto il dataset in train, validation e test set. 

In [9]:
X_train, X_val, y_train, y_val = mf.split(data, test_size = 0.2)
del data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

## Undersampling: Easy Ensemble

L'easy ensemble tramite un metodo unsupervised divide il dataset in diversi dataset undersampled. 

Documentazione easy ensemble: http://glemaitre.github.io/imbalanced-learn/generated/imblearn.ensemble.EasyEnsemble.html#imblearn.ensemble.EasyEnsemble

Esempio applicazio ee: http://glemaitre.github.io/imbalanced-learn/auto_examples/ensemble/plot_easy_ensemble.html?highlight=easy%20ensemble

In [10]:
X_trainres, y_trainres = mf.easy_ensemble(5, X_train, y_train)

In [11]:
X_trainres_df = pd.DataFrame(data = X_trainres[0], columns = cols)
del X_trainres

# LSTM

In [28]:
X_trainres_df

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D10,D15,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,...,DeviceInfo_rv:33.0,DeviceInfo_rv:35.0,DeviceInfo_rv:37.0,DeviceInfo_rv:38.0,DeviceInfo_rv:39.0,DeviceInfo_rv:41.0,DeviceInfo_rv:42.0,DeviceInfo_rv:43.0,DeviceInfo_rv:44.0,DeviceInfo_rv:45.0,DeviceInfo_rv:46.0,DeviceInfo_rv:47.0,DeviceInfo_rv:48.0,DeviceInfo_rv:49.0,DeviceInfo_rv:50.0,DeviceInfo_rv:51.0,DeviceInfo_rv:52.0,DeviceInfo_rv:52.9,DeviceInfo_rv:53.0,DeviceInfo_rv:54.0,DeviceInfo_rv:55.0,DeviceInfo_rv:56.0,DeviceInfo_rv:57.0,DeviceInfo_rv:58.0,DeviceInfo_rv:59.0,DeviceInfo_rv:60.0,DeviceInfo_rv:61.0,DeviceInfo_unknown,DeviceInfo_verykool,DeviceInfo_verykoolS5005,DeviceInfo_verykoolS5019,DeviceInfo_verykoolS5524,DeviceInfo_verykoolS5525,DeviceInfo_verykoolS5530 Build/LMY47I,DeviceInfo_verykools4009,DeviceInfo_verykools5004,DeviceInfo_verykools5034,DeviceInfo_verykools5035,DeviceInfo_vivo,DeviceInfo_xs-Z47b7VqTMxs
0,3189915.0,4646139.0,0.000932,0.482870,0.019429,0.381679,0.585568,0.341741,0.877533,0.000427,0.000351,0.0,0.000000,0.0,0.000888,0.000000,0.000000,0.009524,0.000000,0.000627,0.000000,0.002056,0.001400,0.349633,0.287333,0.526455,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3006650.0,512512.0,0.001072,0.580938,0.000000,0.381679,0.589388,0.418991,0.877533,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.000000,0.004762,0.000000,0.000314,0.000000,0.000343,0.000700,0.027489,0.287333,0.526455,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3027089.0,992152.0,0.001558,0.876926,0.653514,0.381679,0.419398,0.310473,0.877533,0.000427,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.000000,0.000000,0.000000,0.000314,0.000000,0.000685,0.001400,0.027489,0.357596,0.570124,0.409871,0.301649,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3091672.0,2075392.0,0.001699,0.816567,0.300263,0.381679,0.474788,0.224026,0.877533,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.000000,0.004762,0.000000,0.000314,0.000000,0.000000,0.000000,0.029008,0.287333,0.526455,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3000998.0,404992.0,0.000462,0.488618,0.390342,0.381679,0.589388,0.273687,0.877533,0.000213,0.000351,0.0,0.000000,0.0,0.000888,0.000000,0.000000,0.004762,0.000000,0.000314,0.000000,0.002399,0.000700,0.208315,0.307732,0.559981,0.409871,0.301649,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8323,3091222.0,2071211.0,0.004351,0.198954,0.485720,0.6

In [29]:
X = np.expand_dims(X_trainres_df, 1)

In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tensorflow.keras import regularizers


# embedding_vecor_length = 32
model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, input_shape = (None ,X_trainres_df.shape[1]), kernel_initializer='lecun_uniform', 
               activation='relu', kernel_regularizer=regularizers.l1(0.1), 
               recurrent_regularizer=regularizers.l1(0.01), bias_regularizer=None, 
               activity_regularizer=None, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X, y_trainres[0], epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100)               1094800   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 1,094,901
Trainable params: 1,094,901
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
131/131 [==============================] - 6s 28ms/step - loss: 8911.6982 - accuracy: 0.5018
Epoch 2/3
131/131 [==============================] - 4s 29ms/step - loss: 455.4282 - accuracy: 0.5103
Epoch 3/3
131/131 [==============================] - 4s 28ms/step - loss: 136.0431 - accuracy: 0.5143


ValueError: ignored